In [9]:
import setup
setup.init()

In [2]:
config = {
    "configurable": {
        "user_id": "1"
    }
}


In [3]:
from mypermit import permit_client

permit = permit_client

In [4]:
await permit.api.users.assign_role({
    "user": "1",
    "role": "manager",
    "tenant": "default"
})


RoleAssignmentRead(id=UUID('242fb959-46af-44d5-8601-4b95323c200c'), user='1', role='manager', tenant='default', resource_instance=None, resource_instance_id=None, user_id=UUID('b7270928-62cf-455e-905c-d3afe544bffb'), role_id=UUID('9bde09c4-0ff8-4f87-a898-a22cbb44cabe'), tenant_id=UUID('73198a56-5417-432c-8c13-4d21bfe36102'), organization_id=UUID('7613bbbb-aafc-477b-928e-e0d4a3f298ca'), project_id=UUID('d873adca-2b46-4064-a9d5-76d54732c870'), environment_id=UUID('412a05d3-ae9e-444e-9c8d-eef5a5958b51'), created_at=datetime.datetime(2026, 2, 4, 7, 29, 45, tzinfo=datetime.timezone.utc))

In [5]:
from ai.agents import get_agent
from langgraph.checkpoint.memory import InMemorySaver
import uuid

checkpointer = InMemorySaver()

config = {
    "configurable": {
        "user_id": "1",
        "thread_id": str(uuid.uuid4())
    }
}

agent = get_agent(None, checkpointer=checkpointer)


In [6]:
response = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Create a document with title 'Doc1' and content 'New doc'"
            }
        ]
    },
    config
)

response["messages"][-1].content


"I have successfully created a document with the title 'Doc1' and the content 'New doc'."

In [7]:
response = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "list the documents"}
        ]
    },
    config
)

response["messages"][-1].content


'Here are the most recent documents:\n1. Doc1\n2. New Doc\n3. Test Doc\n4. Title-1\n5. Title-2\n\nIf you need more information or want to perform any specific actions on these documents, feel free to let me know!'

In [12]:
response = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Update document 1 and change the title to 'Updated Title'"
            }
        ]
    },
    config
)

print(response["messages"][-1].content)


The document with ID 1 has been successfully updated, and the title has been changed to "Updated Title".


In [13]:
response = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "list the documents"}
        ]
    },
    config
)

print(response["messages"][-1].content)


Here are the most recent documents:
1. New Doc
2. Test Doc
3. Title-1
4. Title-2
5. Title-1


In [14]:
response = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Update document 1 and change the title to 'New Doc Updated'"
            }
        ]
    },
    config
)

print(response["messages"][-1].content)


The document with ID 1 has been successfully updated, and the title has been changed to "New Doc Updated".


In [15]:
response = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "list the documents"}
        ]
    },
    config
)

print(response["messages"][-1].content)


Here are the most recent documents:
1. New Doc
2. Test Doc
3. Title-1
4. Title-2
5. Title-1

The document with the updated title "New Doc Updated" is not reflecting in the list as it was not updated in the recent 5 documents.


In [16]:
response = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "Show document 1"}
        ]
    },
    config
)

print(response["messages"][-1].content)

Here is the document with ID 1:
- Title: New Doc Updated
- Content: test content
- Created At: August 2, 2025, 8:16:09 UTC


In [8]:
result = await permit.api.users.list()
print(type(result))
print(result)


<class 'permit.api.models.PaginatedResultUserRead'>
data=[UserRead(key='2', id=UUID('9b593223-ffb2-4dc1-aa84-f4ff970a4e32'), organization_id=UUID('7613bbbb-aafc-477b-928e-e0d4a3f298ca'), project_id=UUID('d873adca-2b46-4064-a9d5-76d54732c870'), environment_id=UUID('412a05d3-ae9e-444e-9c8d-eef5a5958b51'), associated_tenants=[], roles=[], created_at=datetime.datetime(2025, 8, 14, 14, 13, 12, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 8, 14, 14, 19, 10, tzinfo=datetime.timezone.utc), email=None, first_name='abi', last_name=None, attributes=None), UserRead(key='1', id=UUID('b7270928-62cf-455e-905c-d3afe544bffb'), organization_id=UUID('7613bbbb-aafc-477b-928e-e0d4a3f298ca'), project_id=UUID('d873adca-2b46-4064-a9d5-76d54732c870'), environment_id=UUID('412a05d3-ae9e-444e-9c8d-eef5a5958b51'), associated_tenants=[UserInTenant(tenant='default', roles=['viewer', 'manager'], status=<UserStatus.active: 'active'>, resource_instance_roles=[])], roles=[UserRole(role='viewer', te

In [9]:

result = await permit.api.users.list()

users = result.data  # THIS is the important part

for u in users:
    role_keys = [r.role for r in u.roles]
    print(f"User {u.key} roles:", role_keys)

User 2 roles: []
User 1 roles: ['viewer', 'manager']


In [10]:
from django.contrib.auth import get_user_model
User = get_user_model()

result = await permit.api.users.list()
permit_users = {u.key: u for u in result.data}

for u in User.objects.all():
    pu = permit_users.get(str(u.id))
    roles = [r.role for r in pu.roles] if pu else []
    print(f"Django user {u.username} (id={u.id}) → Permit roles: {roles}")


Django user root (id=1) → Permit roles: ['viewer', 'manager']
Django user abi (id=2) → Permit roles: []
Django user test (id=3) → Permit roles: []


In [ ]:
response = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Create a document with title 'New Doc' and content 'Created now'"
            }
        ]
    },
    config
)

response["messages"][-1].content

In [10]:
response = agent.invoke(
    {"messages" :
        [
            {"role": "user", "content":"Tell me about the first loard of the rings movie"}
        ]
     },
     config
)

response["messages"][-1].content

'I have created a document with information about the first "Lord of the Rings" movie. If you need more details or have any other requests, feel free to let me know!'

In [13]:
from ai.supervisors import get_supervisor
from langgraph.checkpoint.memory import InMemorySaver
import uuid

checkpointer = InMemorySaver()

config = {
    "configurable": {
        "user_id": "1",
        "thread_id": str(uuid.uuid4())
    }
}

agent = get_supervisor(None, checkpointer=checkpointer)

In [15]:
response = agent.invoke(
    {"messages" :
        [
            {"role": "user", "content":"Tell me story about it lord of rings"}
        ]
     },
     config
)

response["messages"][-1].content

Searching for user: 1
Searching for user: 1


'I have provided a summary of "The Lord of the Rings: The Return of the King." If you have any more questions or need further information, feel free to ask!'